# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [6]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import  fetch_california_housing

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier,AdaBoostClassifier, GradientBoostingClassifier

from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

In [7]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [8]:
spaceship.dropna(inplace=True)
spaceship['Cabin'] = spaceship['Cabin'].str[0]
spaceship.drop(['PassengerId','Name'], axis=1, inplace=True)

In [9]:
# Subset data into numerical and categorical
spaceship_cat = spaceship.drop(columns=['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck'])
spaceship_num = spaceship.drop(columns=['HomePlanet','CryoSleep','Cabin','Destination','VIP','Transported'])
target = spaceship["Transported"]

# One Hot Encodes the categorical variables
# Drop if binary drops the first value if there's only two values, otherwise keeps them all - yields .7662 predictive strength
# Drop first just drops first of each categorical - yields .7677 predictive strength
encoder = OneHotEncoder(drop= 'if_binary').fit(spaceship_cat)

# creates column names object 
cols = encoder.get_feature_names_out(input_features=spaceship_cat.columns)

# One Hot Encodes spaceship_cat data, saves as new object
spaceship_encode = pd.DataFrame(encoder.transform(spaceship_cat).toarray(),columns=cols)

# Saves encoded target
target = spaceship_encode['Transported_True']

# Removes target from data that will become feature
spaceship_encode.drop(columns=['Transported_True'], inplace=True)

# Resets indexes in both dfs
spaceship_encode.reset_index(drop=True, inplace=True)
spaceship_num.reset_index(drop=True, inplace=True)

# merges dfs
features = pd.concat([spaceship_encode,spaceship_num],axis=1)

**Perform Train Test Split**

In [10]:
# train test splits our features and target, trains 80% of data, tests 20% of data, sets random state, saves output objects
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=8675309)

# Creates scaler
scaler = MinMaxScaler()

# fits/sets scaler
scaler.fit(X_train)

# scales train and test data and saves as new objects
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# converts the X_train_scaled back into a dataframe using the columns of the original X_train df
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)

# does the same thing for X_test_scaled
X_test_scaled = pd.DataFrame(X_test_scaled, columns = X_test.columns)

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [11]:
bagging_class = BaggingClassifier(DecisionTreeClassifier(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

bagging_class.fit(X_train_scaled, y_train)

pred = bagging_class.predict(X_test_scaled)

# from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

print("Accuracy", accuracy_score(y_test, pred))
print("Recall", recall_score(y_test, pred))
print("Precision", precision_score(y_test, pred, average='binary')) # average = 'binary' is the default

# 0.77 - 0.80

Accuracy 0.7874432677760969
Recall 0.775811209439528
Precision 0.8030534351145038


- Random Forests

In [12]:
forest = RandomForestClassifier(n_estimators=100,
                             max_depth=20)

forest.fit(X_train_scaled, y_train)

pred = forest.predict(X_test_scaled)

print("Accuracy", accuracy_score(y_test, pred))
print("Recall", recall_score(y_test, pred))
print("Precision", precision_score(y_test, pred, average='binary'))

# 0.77 - 0.78

Accuracy 0.7760968229954615
Recall 0.7728613569321534
Precision 0.7867867867867868


- Gradient Boosting

In [13]:
gb_reg = GradientBoostingClassifier(max_depth=20,
                                   n_estimators=100)

gb_reg.fit(X_train_scaled, y_train)

pred = gb_reg.predict(X_test_scaled)

print("Accuracy", accuracy_score(y_test, pred))
print("Recall", recall_score(y_test, pred))
print("Precision", precision_score(y_test, pred, average='binary'))

# 0.76 - 0.77

Accuracy 0.7632375189107413
Recall 0.7728613569321534
Precision 0.767203513909224


- Adaptive Boosting

In [14]:
ada_reg = AdaBoostClassifier(DecisionTreeClassifier(max_depth=20),
                            n_estimators=100)

ada_reg.fit(X_train_scaled, y_train)

pred = ada_reg.predict(X_test_scaled)

print("Accuracy", accuracy_score(y_test, pred))
print("Recall", recall_score(y_test, pred))
print("Precision", precision_score(y_test, pred, average='binary'))

# 0.74 - 0.76

Accuracy 0.7420574886535553
Recall 0.7640117994100295
Precision 0.7410586552217453


Which model is the best and why?

In [ ]:
# It seems that the Bagging and Pasting method is best for this dataset because it yields both the highest accuracy and precision.